In [2]:
from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']

newsgroups_train = fetch_20newsgroups(subset = 'train',
                                      remove = ('headers', 'footers', 'quotes'),
                                      categories = categories)
# 뉴스데이터 사용해서 연습 

In [3]:
from nltk.corpus import stopwords
cachedStopWords = stopwords.words("english")
import nltk
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
import re

In [5]:
def tokenize(text):
    min_length = 3 #최소 단어 크기
    # 모든 문자를 소문자로 변환시켜서 대소문자 구분을 없앤다.
    words = map(lambda word: word.lower(), word_tokenize(text))
    words = [word for word in words if word not in cachedStopWords] # 불용어 제거.
    words_tag = nltk.pos_tag(words) # 쓸데없는 단어가 너무 많이 등장해서 명사만 추출했다.
    nouns = [word for (word, pos) in words_tag if(pos[:2] == 'NN')]
    
    tokens = (list(map(lambda token: PorterStemmer().stem(token), nouns))) 
    # PorterStemmer로 어간만 추출한다. 
    # 이로써 다양한 단어 변형을 하나로 통일한다.
    #tokens = [PorterStemmer().stem(token) for token in nouns]  #이렇게 해도 되는지 확인
    
    p = re.compile('[a-zA-Z]+'); # 원하는 정규표현식 특징을 변수에 할당. 알파벳만 가져온다. 
    # 그런데 이미 모든 문자를 소문자와 시킨 상태에서 A-Z는 필요없지 않나???
    
    filtered_tokens = list(filter (lambda token: p.match(token) and len(token) >= min_length, tokens))
    #filtered_tokens = [token for token in tokens if p.match(token) and len(token) >= min_length]   #이렇게 해도 되는지 확인
    return filtered_tokens 

In [30]:
from sklearn.model_selection import train_test_split
# 훈련 데이터와 테스트 데이터 구분.
X_train, X_test, y_train, y_test = train_test_split(newsgroups_train.data, newsgroups_train.target, random_state = 0)
# 나중에 회귀분석을 위해서 종속변수가 될 데이터를 카테고리 라벨인 newsgroups_train.target으로 했다.

In [18]:
print(tokenize(X_train[1])[:100]) # 토크나이즈하고 명사만 추출할 때

['reader', 'post', 'list', 'church', 'belief', 'warn', 'list', 'sunday', 'sabbath', 'saturday', 'time', 'touch', 'weekend', 'hope']


In [7]:
train_texts = [tokenize(text) for text in X_train]
test_texts = [tokenize(text) for text in X_test]

In [20]:
print('train text counts:', len(train_texts), ', test text counts:', len(test_texts))
print("train text sample", X_train[1])
print("tokenized result", train_texts[1])
print("test text sample", X_test[0])
print("tokenized result", test_texts[1])

# 이렇게 보면 명사만 추출하는 것만으로는 부족해보인다.

train text counts: 1525 , test text counts: 509
train text sample To all a.a readers:
     I have been asked be several of you to post a list of the SDA Church's 
27 Fundamental beliefs.  I warn you now, it's a long list.  However, I'll 
post it on Sunday.  Sabbath is coming up soon so I won't be reading on 
Saturday.  And I don't have time to do it now.
     I would GREATLY appreciate it if you would keep me in touch with what's 
going on.
     I hope all of you have a reastful and relaxing weekend.  I hope it's 
the best one so far!!
tokenized result ['reader', 'post', 'list', 'church', 'belief', 'warn', 'list', 'sunday', 'sabbath', 'saturday', 'time', 'touch', 'weekend', 'hope']
test text sample 
Hi,
  See Roger Grywalski's response to :

Re: Help on network visualization

in comp.graphics.visualization.

Amongst other things, it does exactly this!

tokenized result ['use', 'version', 'work', 'softwar', 'anim', 'quit', 'program', 'thing', 'imho', 'hope', 'author', 'fix']


### word2vec 연습

In [25]:
from gensim.models import word2vec

wv_model = word2vec.Word2Vec(train_texts)
wv_model.init_sims(replace = True)

In [28]:
#유사한 단어 추출
print(wv_model.wv.most_similar('softwar'))

# 어간 추출 후 상태가 이상하다.

[('packag', 0.9998846650123596), ('graphic', 0.9997982978820801), ('program', 0.9997957348823547), ('process', 0.9997274875640869), ('data', 0.9996607303619385), ('display', 0.9996533393859863), ('tool', 0.9995932579040527), ('directori', 0.9995844960212708), ('ftp', 0.9995650053024292), ('site', 0.999546468257904)]


In [30]:
# 다른 단어들과 가장 거리가 먼 단어
print(test_texts[10])
wv_model.wv.doesnt_match(test_texts[10])

['thank', 'sound', 'regroup', 'strategi', 'session', 'brochur', 'letter', 'invit', 'space', 'program', 'develop', 'advanc', 'attend', 'wow', 'wonder', 'contribut', 'space', 'letter', 'hope', 'pat', 'net', 'use', 'eye', 'ear']


'brochur'

### doc2vec 연습

In [8]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# doc2vec 학습을 위해서는 각 tokenize된 document에 tag를 붙여서 TaggedDocument를 만들어야 함
# tokenized documents인 train_texts에 대해 아래 식을 그대로 적용
tagged_data = [TaggedDocument(doc, [i]) for i, doc in enumerate(train_texts)]

#모형 정의 및 학습
dv_model = Doc2Vec(tagged_data)

In [9]:
#학습된 모형으로 tokenized text를 벡터로 변환
dv_train = [dv_model.infer_vector(doc) for doc in train_texts]
dv_test = [dv_model.infer_vector(doc) for doc in test_texts]

In [10]:
dv_train

[array([-0.00819182, -0.03632132, -0.01768492, -0.02476414,  0.0145094 ,
         0.00318285,  0.01251268, -0.01336267,  0.02549392, -0.01173957,
        -0.00049437,  0.02256677,  0.00064664,  0.02732459, -0.00252089,
        -0.00387052, -0.04206527,  0.02837194,  0.01110537,  0.00751107,
         0.01417048,  0.00731715,  0.02413292, -0.01392323, -0.00370403,
        -0.00203335, -0.01400691,  0.00398968,  0.00380788, -0.01140567,
        -0.01209844,  0.03456824,  0.02021501,  0.04154024, -0.03767643,
         0.01744425,  0.00239277, -0.00680795,  0.00566195,  0.0102773 ,
         0.03090816,  0.00670416,  0.00991495, -0.0033561 , -0.01474777,
        -0.00515406,  0.00638321, -0.0185302 , -0.01999046,  0.02405143,
        -0.00909197,  0.00997961,  0.00054564, -0.01515364, -0.01364768,
         0.01179809,  0.0170571 , -0.00908518,  0.00381637,  0.01498948,
         0.01809953,  0.02289877, -0.00465702, -0.00902588, -0.01306294,
        -0.01934847,  0.00284311, -0.00034643, -0.0

In [11]:
len(dv_train)

1525

In [32]:
#변환된 vector를 이용하여 로지스틱 회귀분석을 실행
from sklearn.linear_model import LogisticRegression 

clf = LogisticRegression() #분류기 선언
clf.fit(dv_train, y_train) # train data를 이용하여 분류기를 학습
print('Train set score: {:.3f}'.format(clf.score(dv_train, y_train))) # train data에 대한 예측정확도 
print('Test set score: {:.3f}'.format(clf.score(dv_test, y_test))) # test data에 대한 예측정확도



Train set score: 0.476
Test set score: 0.444


C:\Users\Namlister\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Namlister\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


예측률이 너무 낮게 나온다. ???

In [33]:
#변환된 vector들을 normalize하여 로지스틱 회귀분석을 실행
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler() # 검색해보니까 scikit-learn의 전처리 기능 중 하나이다.
# StandardScaler()는 평균이 0과 표준편차가 1이 되도록 변환한다. 
# 수학공부

sc_train = scaler.fit_transform(dv_train)
sc_test = scaler.transform(dv_test)

clf.fit(sc_train, y_train) # train data를 이용하여 분류기를 학습
print('Train set score: {:.3f}'.format(clf.score(sc_train, y_train))) # train data에 대한 예측정확도 
print('Test set score: {:.3f}'.format(clf.score(sc_test, y_test))) # test data에 대한 예측정확도


# 원래라면 정규화 과정도 우선, train 데이터를 입력으로 하여 fit 메서드를 실행하면 분포 모수를 객체내에 저장한다.
# train 데이터를 입력으로 하여 transform 메서드를 실행하면 train 데이터를 변환
# test 데이터를 입력으로 하여 transform 메서드를 실행하면 test 데이터를 변환
# train 데이터의 경우 그냥 한 번에 fit_transform 메서드를 사용할 수도 있다. 위에서는 fit_transform을 사용한 것을 볼 수 있다.
# https://datascienceschool.net/view-notebook/f43be7d6515b48c0beb909826993c856/ 



#normalize 전보다 결과가 나아진 것을 볼 수 있음

Train set score: 0.621
Test set score: 0.521


C:\Users\Namlister\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Namlister\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


여전히 결과는 마음에 안들지만 조금 나아진 모습을 보인다.

In [34]:
# 첫째 문서와 가장 유사한 문서를 찾아보자.
dv_model.docvecs.most_similar(0)

[(494, 0.9776981472969055),
 (191, 0.9757053256034851),
 (1261, 0.9755437970161438),
 (980, 0.9755029082298279),
 (1228, 0.9753338098526001),
 (1284, 0.975281298160553),
 (2, 0.9752587080001831),
 (205, 0.9752233028411865),
 (602, 0.9750911593437195),
 (729, 0.9750728607177734)]

In [35]:
X_train[0] # 이게 첫째 문서이다.

'\t[...deleted...]\n\nUnfortunately, I think you\'ve got it figured pretty well.  I also ask\nmyself the question "Why did they plan for so many months.  Why was\nthis so important to them?  What was the government really up to?\nWhy did they seal the warrant?  Were they after Koresh or were they after      \nthe first and second amendments, among others?\n'

In [36]:
# 가장 유사하다는 결과를 보이는 494번째 문서를 보자.
X_train[494]

'\n\n\n\nMy argument is mainly a proposal of what I think is a plausible argument\nagainst extra-marital sex -- one which I personally believe has some\ntruth.  My main purpose for posting it here is to show that a\n_plausible_ argument can be made against extra-marital sex.  At this\nstage I am not saying that this particular viewpoint is proven or\nanything like that, just that it is plausible.  To try to convince you\nall of this particular point of view, I would probably have to do a lot\nof work researching what has been done in this field, etc., in order to\ngather further evidence, which I simply do not have time to do now.  \n\nAlso note that I said that I think extra-marital sex is "a prime cause"\n(in my opinion) of the generally greater levels of psychological\nproblems, especially depression, in Western societies.  I am not saying\nit is "the prime cause" or "the only cause", just "a prime cause" --\ni.e. one of the significant contributions to this trend.  I think when\nyo

솔직히 어느 부분에서 유사하다는지 이유를 잘 모르겠다. 토크나이즈 할 때 명사만 추출해서 그런가